<a href="https://colab.research.google.com/github/gjrkdgh0772/Finance/blob/main/lec09_%ED%8E%80%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6><b>Lec08. 파생상품-펀드(Fund)
* ref : KRX 아카데미,  http://academy.krx.co.kr/contents/ACA/02/02020200/ACA02020200.jsp
* ref : 키움증권 ,     https://www.kiwoom.com/h/help/trade/VHelpFuopInfoView
* ref : 금융공학레시피, http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791162240823
* ref : KRX 정보데이터시스템, http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201050101

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)


# 시가총액 상위 종목
* http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0302
* 쉽게보는통계 > 순위통계 > 주식 > 시가총액상하위50종목

In [ ]:
top_df = pd.read_csv("/content/drive/MyDrive/금융/시가총액_202001.csv", encoding='cp949')
top_df.head(10)

,순위,종목코드,종목명,시장구분,종가,대비,등락률,거래량,거래대금,시가총액,시가총액비중,상장주식수
0,1,5930,삼성전자,KOSPI,83000,2000,2.47,38655276,3185356823460,495491951650000,20.44,5969782550
1,2,660,SK하이닉스,KOSPI,126000,7500,6.33,7995016,994276505704,91728297990000,3.78,728002365
2,3,51910,LG화학,KOSPI,889000,65000,7.89,858451,747929748128,62756592927000,2.59,70592343
3,4,5935,삼성전자우,KOSPI,74400,800,1.09,5455139,405685236800,61222770480000,2.53,822886700
4,5,207940,삼성바이오로직스,KOSPI,829000,3000,0.36,182864,149889473000,54850785000000,2.26,66165000
5,6,35420,NAVER,KOSPI,293000,500,0.17,947178,274635759500,48129174735000,1.99,164263395
6,7,68270,셀트리온,KOSPI,347500,-11500,-3.20,2006483,698238748000,46911737237500,1.94,134997805
7,8,6400,삼성SDI,KOSPI,671000,43000,6.85,694737,460597285668,46140999630000,1.90,68764530
8,9,5380,현대차,KOSPI,207500,15500,8.07,5651695,1162238349000,44336148802500,1.83,213668187
9,10,35720,카카오,KOSPI,396000,6500,1.67,853740,335358065000,35020876176000,1.44,88436556


In [ ]:
print(str(5930).zfill(6))

005930


In [ ]:
top_df['종목코드2'] = top_df['종목코드'].apply(lambda x : str(x).zfill(6))
top_df.head(10)

,순위,종목코드,종목명,시장구분,종가,대비,등락률,거래량,거래대금,시가총액,시가총액비중,상장주식수,종목코드2
0,1,5930,삼성전자,KOSPI,83000,2000,2.47,38655276,3185356823460,495491951650000,20.44,5969782550,005930
1,2,660,SK하이닉스,KOSPI,126000,7500,6.33,7995016,994276505704,91728297990000,3.78,728002365,000660
2,3,51910,LG화학,KOSPI,889000,65000,7.89,858451,747929748128,62756592927000,2.59,70592343,051910
3,4,5935,삼성전자우,KOSPI,74400,800,1.09,5455139,405685236800,61222770480000,2.53,822886700,005935
4,5,207940,삼성바이오로직스,KOSPI,829000,3000,0.36,182864,149889473000,54850785000000,2.26,66165000,207940
5,6,35420,NAVER,KOSPI,293000,500,0.17,947178,274635759500,48129174735000,1.99,164263395,035420
6,7,68270,셀트리온,KOSPI,347500,-11500,-3.20,2006483,698238748000,46911737237500,1.94,134997805,068270
7,8,6400,삼성SDI,KOSPI,671000,43000,6.85,694737,460597285668,46140999630000,1.90,68764530,006400
8,9,5380,현대차,KOSPI,207500,15500,8.07,5651695,1162238349000,44336148802500,1.83,213668187,005380
9,10,35720,카카오,KOSPI,396000,6500,1.67,853740,335358065000,35020876176000,1.44,88436556,035720


In [ ]:
str(top_df.loc[0, '종목명'])[-1]

'자'

In [ ]:
top10_name = top_df.loc[:10, '종목명'].values
top10_name

array(['삼성전자', 'SK하이닉스', 'LG화학', '삼성전자우', '삼성바이오로직스', 'NAVER', '셀트리온',
       '삼성SDI', '현대차', '카카오', '현대모비스'], dtype=object)

In [ ]:
top10_ticker = top_df.loc[:10, '종목코드2'].values
top10_ticker

array(['005930', '000660', '051910', '005935', '207940', '035420',
       '068270', '006400', '005380', '035720', '012330'], dtype=object)

# 유통주식수 가져오기 _ DART OpenAPI 이용
* ref : https://github.com/FinanceData/OpenDartReader
* ref : https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS002&apiId=2020002


In [ ]:
# ! pip install dart-fss
# ! pip install opendartreader

In [ ]:
import requests
import json
import OpenDartReader
try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize

API_KEY = "e1c8d2f41a9bdb8e5001ecbfa410740d001b1969"

res = requests.get(f"https://opendart.fss.or.kr/api/stockTotqySttus.json?crtfc_key={API_KEY}&corp_code=00126380&bsns_year=2019&reprt_code=11011")
res = json.loads(res.text)
print(type(res), res)

res_df = pd.json_normalize(res, 'list')
res_df.head()

<class 'dict'> {'status': '000', 'message': '정상', 'list': [{'rcept_no': '20200330003851', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'se': '보통주', 'isu_stock_totqy': '20,000,000,000', 'now_to_isu_stock_totqy': '7,780,466,850', 'now_to_dcrs_stock_totqy': '1,810,684,300', 'redc': '-', 'profit_incnr': '1,810,684,300', 'rdmstk_repy': '-', 'etc': '-', 'istc_totqy': '5,969,782,550', 'tesstk_co': '-', 'distb_stock_co': '5,969,782,550'}, {'rcept_no': '20200330003851', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'se': '우선주', 'isu_stock_totqy': '5,000,000,000', 'now_to_isu_stock_totqy': '1,194,671,350', 'now_to_dcrs_stock_totqy': '371,784,650', 'redc': '-', 'profit_incnr': '371,784,650', 'rdmstk_repy': '-', 'etc': '-', 'istc_totqy': '822,886,700', 'tesstk_co': '-', 'distb_stock_co': '822,886,700'}, {'rcept_no': '20200330003851', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'se': '합계', 'isu_stock_totqy': '25,000,000,000', 'now_to_isu_stock_t

,rcept_no,corp_cls,corp_code,corp_name,se,isu_stock_totqy,now_to_isu_stock_totqy,now_to_dcrs_stock_totqy,redc,profit_incnr,rdmstk_repy,etc,istc_totqy,tesstk_co,distb_stock_co
0,20200330003851,Y,00126380,삼성전자,보통주,"20,000,000,000","7,780,466,850","1,810,684,300",-,"1,810,684,300",-,-,"5,969,782,550",-,"5,969,782,550"
1,20200330003851,Y,00126380,삼성전자,우선주,"5,000,000,000","1,194,671,350","371,784,650",-,"371,784,650",-,-,"822,886,700",-,"822,886,700"
2,20200330003851,Y,00126380,삼성전자,합계,"25,000,000,000","8,975,138,200","2,182,468,950",-,"2,182,468,950",-,-,"6,792,669,250",-,"6,792,669,250"
3,20200330003851,Y,00126380,삼성전자,비고,-,-,-,-,자사주소각,-,-,-,-,-


In [ ]:
# istc_totqy     발행주식의 총수
# tesstk_co      자기주식수
# distb_stock_co 유통주식수
total_amount = res_df.head().iloc[2]['distb_stock_co'].replace(",","") # 문자열의 , 를 제거한다.
print(total_amount)

6792669250


In [ ]:
# ! pip install --upgrade pandas==1.3.5

In [ ]:
API_KEY = "e1c8d2f41a9bdb8e5001ecbfa410740d001b1969"

import OpenDartReader
dart = OpenDartReader(API_KEY)
dart.major_shareholders('삼성전자')

# import dart_fss as dart
# dart.set_api_key(api_key=API_KEY)


,rcept_no,rcept_dt,corp_code,corp_name,report_tp,repror,stkqy,stkqy_irds,stkrt,stkrt_irds,ctr_stkqy,ctr_stkrt,report_resn
0,20210503000505,2021-05-03,00126380,삼성전자,일반,삼성물산,"1,263,039,758","-3,582,011",21.16,-0.06,"88,577,273",1.48,- 특별관계자 및 보유주식수 변동\n- 보유주식등에 관한 계약의 체결
1,20210504000163,2021-05-04,00126380,삼성전자,약식,국민연금공단,"578,352,347","-59,717,656",9.69,-1.00,-,-,단순추가취득/처분
2,20210507000809,2021-05-07,00126380,삼성전자,일반,삼성물산,"1,263,093,732","53,974",21.16,0.00,"88,577,273",1.48,- 특별관계자 및 보유주식수 변동
3,20210730000519,2021-07-30,00126380,삼성전자,일반,삼성물산,"1,262,820,989","-272,743",21.15,-0.01,"88,577,273",1.48,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
4,20210806000286,2021-08-06,00126380,삼성전자,일반,삼성물산,"1,262,889,978","68,989",21.15,0.00,"88,577,273",1.48,- 특별관계자 및 보유주식수 변동
5,20210924000389,2021-09-24,00126380,삼성전자,일반,삼성물산,"1,262,644,773","-245,205",21.15,0.00,"96,884,930",1.62,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
6,20211001000488,2021-10-01,00126380,삼성전자,일반,삼성물산,"1,262,589,790","-54,983",21.15,0.00,"94,292,587",1.58,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
7,20211008000519,2021-10-08,00126380,삼성전자,일반,삼성물산,"1,262,563,584","-26,206",21.15,-0.00,"114,234,447",1.91,- 보유주식수 변동\n- 유가증권신탁(보유형태 변경 포함)
8,20211029000589,2021-10-29,00126380,삼성전자,일반,삼성물산,"1,262,501,732","-61,852",21.15,-0.00,"116,766,447",1.96,- 보유주식수 변동\n- 보유주식등에 관한 계약의 변경
9,20211103000100,2021-11-03,00126380,삼성전자,약식,국민연금공단,"518,509,578","-59,842,769",8.69,-1.00,-,-,단순추가취득/처분


# OHLCV 가져오기

<img src="https://img.hankyung.com/photo/202001/AA.21406035.1.jpg">

In [ ]:
# ! pip install -U finance-datareader
# ! pip install finance-datareader
# ! pip install pykrx
# ! pip install bokeh
# ! pip install yfinance
# ! pip install pandas_datareader

In [ ]:
# ! pip install plotly

## pykrx

In [ ]:
from pykrx import stock, bond
# 2018-04-27(거래정지전마지막거래일) 2,650,000
# 2018-05-04(액면분할후거래시작)       51,900
df = stock.get_market_ohlcv('2018-04-25', '2018-05-06', "005930")
df.head(10)

,시가,고가,저가,종가,거래량
날짜,,,,,
2018-04-25,49220,50500,49220,50400,332292
2018-04-26,50420,52160,50400,52140,360931
2018-04-27,53380,53639,52440,53000,606216
2018-04-30,0,0,0,53000,0
2018-05-02,0,0,0,53000,0
2018-05-03,0,0,0,53000,0
2018-05-04,53000,53900,51800,51900,39565391


## FinanceDataReader

In [ ]:
import FinanceDataReader as fdr
df = fdr.DataReader('005930', '2018-04-25', '2018-05-06', exchange='KRX') 
df.head(10)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-04-25,49220,50500,49220,50400,332292,-0.001189
2018-04-26,50420,52160,50400,52140,360931,0.034524
2018-04-27,53380,53639,52440,53000,606216,0.016494
2018-04-30,0,0,0,53000,0,0.000000
2018-05-02,0,0,0,53000,0,0.000000
2018-05-03,0,0,0,53000,0,0.000000
2018-05-04,53000,53900,51800,51900,39565391,-0.020755


## yfinance

In [ ]:
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
df = pdr.get_data_yahoo('005930.KS','2018-04-25', '2018-05-06')
df.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-04-25,49220.0,50500.0,49220.0,50400.0,44856.070312,16614600
2018-04-26,50420.0,52160.0,50400.0,52140.0,46404.671875,18046550
2018-04-27,53380.0,53640.0,52440.0,53000.0,47170.074219,30310800
2018-04-30,53000.0,53000.0,53000.0,53000.0,47170.074219,0
2018-05-02,53000.0,53000.0,53000.0,53000.0,47170.074219,0
2018-05-03,53000.0,53000.0,53000.0,53000.0,47170.074219,0
2018-05-04,53000.0,53900.0,51800.0,51900.0,46191.078125,39565391


In [ ]:
# 시가총액 상위 10개종목 :  업체명, 발행주식수, 유동비율 프레임화
# 시가총액계산 = 일별주가*발행주식수*(유동비율%*0.01)
# 일자별 10개종목의 시가총액계산(총합) 2017-01-02기준 index지수화
# 코스피200지수
# - 두지수 차트로 비교